In [13]:
from pathlib import Path
import pandas as pd

In [40]:
main_directory = Path(__vsc_ipynb_file__).parents[2]
transporation_emissions_file = main_directory.joinpath('references/background_data/a4_emissions.xlsx')
transportation_distances_file = main_directory.joinpath('references/background_data/a4_distances.xlsx')
bom_file = main_directory.joinpath('data/template_models/STR1_ENCO1_ENCT1_ENCR1/bom/STR1_ENCO1_ENCT1_ENCR1_bom.csv')

background_dataset = pd.read_excel(transporation_emissions_file)
background_distances = pd.read_excel(transportation_distances_file)
bom = pd.read_csv(bom_file)
impacts_map = {
    'Global Warming Potential_fossil': 'GWPf',
    'Global Warming Potential_biogenic': 'GWPb',
    'Global Warming Potential_luluc': 'GWP-LULUC',
    'Acidification Potential': 'acp',
    'Eutrophication Potential': 'eup',
    'Smog Formation Potential': 'smg',
    'Ozone Depletion Potential': 'odp'
}
mi_to_km_conversion = 1.60934

truck_dict = {
    'emissions_name': 'Transport, combination truck, average fuel mix',
    'distance_column': 'Tally dist_truck',
}

trans_emissions = background_dataset.set_index('Product system name')
trans_distances = background_distances[['Name_Tally Material', truck_dict.get('distance_column')]]
# emission = mass of product * emission factor * distance * return factor
temp_df = bom.merge(
    trans_distances,
    left_on='Tally material',
    right_on='Name_Tally Material',
    how='left'
).drop(
    "Name_Tally Material",
    axis=1
)
for name, col_name in impacts_map.items():
    temp_df[name] = (temp_df['Weight (kg)'] / 1000) * trans_emissions.loc[truck_dict.get('emissions_name'), col_name] * (temp_df[truck_dict.get('distance_column')] * mi_to_km_conversion)
    temp_df.loc[temp_df[truck_dict.get('distance_column')] > 500, name] =  1.5 * (temp_df['Weight (kg)'] / 1000) * trans_emissions.loc[truck_dict.get('emissions_name'), col_name] * (temp_df[truck_dict.get('distance_column')] * mi_to_km_conversion)

temp_df.to_csv('truck.csv')

